# Apply model to sources

This notebook ...

# Input

In [64]:
import pandas as pd
import numpy as np
from ENID.aquisition import *
from alerce.core import Alerce
from time import gmtime, strftime
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'ENID'

In [46]:
datasources = pd.read_csv('sources.csv',header=None) # Give your sources in a .csv file with the ZTF-names. An example with blue continuum sources is attached

FileNotFoundError: File b'sources.txt' does not exist

In [80]:
# Name of the saved files
Version = strftime("%Y-%m-%d_%H:%M:%S",gmtime()) #Default used UTC time and date.

# Number of points before
timediff = 14
noofpoint = 2

# Plotting?
plotbool = 'y'

In [77]:
Version

'2021-12-17_16:15:05'

# Apply model

In [60]:
##### Retrieve light curves part

# Get ready by making it a list anc checking that they are not floats
ztf_raw = list(sources[0])
ztf_names = [x for x in ztf_raw if type(x) != float]

# Find the sources on ALeRCE
print('Number of sources :', len(ztf_raw), '\n')
alerce_found, alerce_missing = source_search_alerce(ztf_names)

# Tell how many were found
print('Found ', alerce_found.shape[0], 'objects')
print('Missing ', len(alerce_missing), 'objects')

if len(alerce_missing) > 0:
    for mis in alerce_missing:
        print('Missing ', alerce_missing['oid'][mis])

# Get the object id for each source
sources = list(alerce_found['oid'])

# Retrieve the lightcurves for the sources
object_dictionary = {"Name": [], "Data": [], "Label": []}
for i in range(len(sources)):
    lightcurves = lc_compile(sources[i])
    

Number of sources : 1 



NameError: name 'source_search_alerce' is not defined

In [ ]:
##### Adding in non-detections & interpolating the light curves


processed_data = preprocessing(,Version,timediff,noofpoint)



In [81]:
##### Plot lightcuves AND interpolated light curves

if plotbool == 1 or "Yes" or 'y' or 'Y':
    for scs in processed_data:
        fig, ax = plt.subplots(1,2)
        fig.suptitle("NAME")
        # Real data
        ax[0,0].plt.scatter(datadict['Data'][index]['R_mjd'], datadict['Data'][index]['R_mag'], 60, color=[(232/256, 63/256, 72/256)], label='R-band')
        ax[0,0].plt.scatter(datadict['Data'][index]['G_mjd'], datadict['Data'][index]['G_mag'], 60, color=[(31/256, 208/256, 130/256)], label='G-band')
        ax[0,0].plt.xlabel('Time [MJD]', fontsize=20)
        ax[0,0].plt.ylabel('Apparent Magnitude', fontsize=20)
        ax[0,0].plt.gca().invert_yaxis()
        ax[0,0].plt.grid()
        ax[0,0].plt.legend()
        ax[0,0].plt.title(datadict['Label'][index][0], fontsize=20)
        # Interpolated data
        ax[0,1].plt.scatter(datadict['Data'][index]['R_mjd'], datadict['Data'][index]['R_mag'], 60, color=[(232/256, 63/256, 72/256)], label='R-band')
        ax[0,1].plt.scatter(datadict['Data'][index]['G_mjd'], datadict['Data'][index]['G_mag'], 60, color=[(31/256, 208/256, 130/256)], label='G-band')
        ax[0,1].plt.xlabel('Normalized time', fontsize=20)
        ax[0,1].plt.ylabel('Normalized Flux', fontsize=20)
        ax[0,1].plt.gca().invert_yaxis()
        ax[0,1].plt.grid()
        ax[0,1].plt.legend()
        ax[0,1].plt.title(datadict['Label'][index][0], fontsize=20)
        plt.show()
        
        
    
    
    

NameError: name 'processed_data' is not defined

In [ ]:
##### Classify it


#???

# Function

In [ ]:
def ApplyENID(datasources,Version,timediff,noofpoints,plotbool):

        
    
    